## Initial Search and Explore Search Results

In [1]:
# imports
import requests
import json
import math
import pandas as pd
import spacy

In [2]:
# initial search
url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext=Vietnam&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic&format=json'
response = requests.get(url)
raw = response.text
results = json.loads(raw)

In [3]:
# explore results
results.keys()

dict_keys(['totalItems', 'endIndex', 'startIndex', 'itemsPerPage', 'items'])

In [4]:
# explore items
print(type(results['items']))

<class 'list'>


In [5]:
# expore first item
print(results['items'][0])

{'sequence': 8, 'county': ['Saint Thomas'], 'edition': None, 'frequency': 'Daily (except Sunday)', 'id': '/lccn/sn83025327/1963-10-03/ed-1/seq-8/', 'subject': ['Charlotte Amalie (United States Virgin Islands)--Newspapers.', 'United States Virgin Islands--Charlotte Amalie.--fast--(OCoLC)fst01861757'], 'city': ['Charlotte Amalie'], 'date': '19631003', 'title': 'The Daily news of the Virgin Islands. [volume]', 'end_year': 1978, 'note': ['"The only independent Virgin Islands newspaper serving the people of the Virgin Islands."', '40th anniversary ed., Aug. 1, 1970; 160 pp., 45th anniversary ed., Aug. 1, 1975. Contained ill. history of the islands and newspaper publishing in the territory.', 'Editor: Ariel Melchior, Sr., 1955-1964; Ariel Melchior, Jr., 1964-1976; John Thompson, Jan.-Apr. 1977; Carter Hague, Apr. 1977-May 1978.'], 'state': ['Virgin Islands'], 'section_label': '', 'type': 'page', 'place_of_publication': 'Charlotte Amalie, St. Thomas, V.I.', 'start_year': 1955, 'edition_label'

In [6]:
print('totalItems:', results['totalItems'])
print('endIndex:', results['endIndex'])
print('startIndex:', results['startIndex'])
print('itemsPerPage:', results['itemsPerPage'])
print('Length and type of items:', len(results['items']), type(results['items']))

totalItems: 2353
endIndex: 20
startIndex: 1
itemsPerPage: 20
Length and type of items: 20 <class 'list'>


In [7]:
# find total amount of pages
total_pages = math.ceil(results['totalItems'] / results['itemsPerPage'])
print(total_pages)

118


## Gather Search Results

In [8]:
# create empty list for data
data = []

In [9]:
# set search parameters
start_date = '1770'
end_date = '1865'
search_term = 'Vietnam'

In [12]:
# loop through search results and collect data
for i in range(1, total_pages + 1):  # for sake of time I'm doing only 10, you will want to put total_pages+1
    url = (f'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1={start_date}'
           f'&date2={end_date}&proxtext={search_term}&x=16&y=8&dateFilterType=yearRange&rows=20'
           f'&searchType=basic&format=json&page={i}')  # f-string
    response = requests.get(url)
    raw = response.text
    print(f'page {i} status code:', response.status_code)  # checking for errors
    results = json.loads(raw)
    items_ = results['items']
    for item_ in items_:
        row_data = {}
        try:
          row_data['title'] = item_['title_normal']
        except:
          row_data['title'] = "none"
        try:
          row_data['city'] = item_['city']
        except:
          row_data['city'] = "none"
        try:
          row_data['date'] = item_['date']
        except:
          row_data['date'] = "none"
        try:
          row_data['raw_text'] = item_['ocr_eng']
        except:
          row_data['raw_text'] = 'none'
    data.append(row_data)

page 1 status code: 200
page 2 status code: 200
page 3 status code: 200
page 4 status code: 200
page 5 status code: 200
page 6 status code: 200
page 7 status code: 200
page 8 status code: 200
page 9 status code: 200
page 10 status code: 200
page 11 status code: 200
page 12 status code: 200
page 13 status code: 200
page 14 status code: 200
page 15 status code: 200
page 16 status code: 200
page 17 status code: 200
page 18 status code: 200
page 19 status code: 200
page 20 status code: 200
page 21 status code: 200
page 22 status code: 200
page 23 status code: 200
page 24 status code: 200
page 25 status code: 200
page 26 status code: 200
page 27 status code: 200
page 28 status code: 200
page 29 status code: 200
page 30 status code: 200
page 31 status code: 200
page 32 status code: 200
page 33 status code: 200
page 34 status code: 200
page 35 status code: 200
page 36 status code: 200
page 37 status code: 200
page 38 status code: 200
page 39 status code: 200
page 40 status code: 200
page 41 s

In [13]:
# put data into DataFrame
df = pd.DataFrame.from_dict(data)

In [14]:
# sanity check
df.head()

,title,city,date,raw_text
0,washington statesman.,[Walla Walla],18630131,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."
1,washington statesman.,[Walla Walla],18630131,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."
2,new york herald.,[New York],18580811,DB SUMMER RETREAT8.~ !\nOmjt M?uU?kl CotrMpuo^...
3,evansville daily journal.,[Evansville],18651013,"1.\n4, . ' .\n,\n.\n.17\n,\nI\n4,\n1\nf\n,\n.\..."
4,washington statesman.,[Walla Walla],18630131,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."


### Change Date Format

In [15]:
# convert date column from string to date-time object
df['date'] = pd.to_datetime(df['date'])

In [16]:
# sanity check
df.head()

,title,city,date,raw_text
0,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."
1,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."
2,new york herald.,[New York],1858-08-11,DB SUMMER RETREAT8.~ !\nOmjt M?uU?kl CotrMpuo^...
3,evansville daily journal.,[Evansville],1865-10-13,"1.\n4, . ' .\n,\n.\n.17\n,\nI\n4,\n1\nf\n,\n.\..."
4,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."


In [17]:
# sort by date
df = df.sort_values(by='date')

In [18]:
# sanity check
df.head()

,title,city,date,raw_text
2,new york herald.,[New York],1858-08-11,DB SUMMER RETREAT8.~ !\nOmjt M?uU?kl CotrMpuo^...
0,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."
86,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."
85,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."
84,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,..."


### Process search results

In [19]:
# write fuction to process text
# load nlp model
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner', 'parser')  # these are unnecessary for the task at hand

def process_text(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    lemmas = [token.lemma_ for token in no_punct]
    lemmas_lower = [lemma.lower() for lemma in lemmas]
    lemmas_string = ' '.join(lemmas_lower)
    return lemmas_string

In [20]:
# apply process_text function
# this may take a few minutes
df['lemmas'] = df['raw_text'].apply(process_text)

In [21]:
df.head()

,title,city,date,raw_text,lemmas
2,new york herald.,[New York],1858-08-11,DB SUMMER RETREAT8.~ !\nOmjt M?uU?kl CotrMpuo^...,db summer omjt tiir dinboana norm movtuai augu...
0,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,...",washington statesman momenta bepreciation lost...
86,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,...",washington statesman momenta bepreciation lost...
85,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,...",washington statesman momenta bepreciation lost...
84,washington statesman.,[Walla Walla],1863-01-31,"THE WASHINGTON STATESMAN.\néAitr‘ESLY momenta,...",washington statesman momenta bepreciation lost...


In [22]:
# save to csv
df.to_csv(f'{search_term}{start_date}-{end_date}.csv', index=False)